In [1]:
import networkx as nx 
import numpy as np
import json
import pandas as pd
from sage.graphs.graph_input import from_dict_of_lists
from time import time

In [2]:
def nakljucni_MIS(G, P):
    I = []            #Nastajajoca neodvisna mnozica
    V = list(G.nodes)
    for i in V:        # V tej mnozici izberemo vozlisca, ki jih dodamo neodvisni mnozici. P je permutacija naravnih stevil do vkljucno len(V). Vozlisce dodamo I, ce ima najmanjso vrednost med sosedi.,tako tudi preprecimo ponavljanje vozlisc v mnozici.
        A = []      
        A.append(P[i])
        if list(G.neighbors(i)) != []:
            for j in list(G.neighbors(i)):
                A.append(P[j])
        if P[i] == min(A):
            I.append(i)
    N = [] 
    for i in I:
        N += G.neighbors(i)
    V1 = I + N
    V1 = list(dict.fromkeys(V1)) # V1 je mnozica, ki jo bomo odstranili iz grafa in je sestavljena iz vozlisc iz I, ter njihovih sosedov
    if I == []:
        return []
    else:
        G1 = G.copy()
        G1.remove_nodes_from(V1)
        return list(I + nakljucni_MIS(G1, P))

In [3]:
def tightness(graf, mnozica, v): #Dolocimo "tightness" vozlisca v, v mnozici X, tj. za vsakega soseda vozlisca v, v mnozici graf\X, tightness(v) povecamo za 1
    t = 0
    for w in graf.neighbors(v):
        if w in mnozica:
            t += 1
        else:
            pass
    return t

def lokalno_iskanje(G, I): #Parametra graf v networkx formatu, neodvisna množica I (gre za Python list)
    V = list(G.nodes)
    for v in I:
        L = [] #Gradimo mnozice potencialnih kandidatov za dodajanje v neodvisno mnozico
        for w in G.neighbors(v):
            if G.neighbors(v) != []:
                if tightness(G, I, w) == 1:
                    L.append(w)
                else:
                    pass
            else:
                pass
        if len(L) >= 2:
            for v1 in L:
                for w1 in L:
                    if w1 not in G.neighbors(v1):
                        I.remove(v)
                        I_lokalno = I + [v1, w1]
                        return lokalno_iskanje(G, I_lokalno)
        else:
            return I 

In [4]:
def CLP(G):
    p = MixedIntegerLinearProgram(maximization=True)
    x = p.new_variable(binary=True)
    p.set_objective(sum(x[v] for v in G))

    for u, v in G.edges(labels=False):
        p.add_constraint(x[u] + x[v] <= 1)
    
    p.solve()
    x = p.get_values(x)
    return [v for v, i in x.items() if i]

In [5]:
def zapisi_v_json(datoteka,A):
    with open(datoteka +'.json','w') as js:
        js.write(
            '[' +
            ',\n'.join(json.dumps(i) for i in A) +
            ']\n')

# Dano verjetnost in n
def erdos_renyi(n,p,m):
    A = []
    for i in range(m):
        G = nx.erdos_renyi_graph(n,p)
        G_slovar = nx.to_dict_of_lists(G)
        A += [G_slovar]

    return A


# Razlicne n

def erdos_renyi_N(N, p):
    A = []
    for i in N:
        G = nx.erdos_renyi_graph(i,p)
        G_slovar = nx.to_dict_of_lists(G)
        A += [G_slovar]

    return A


#Razlicne p

def erdos_renyi_P(n, P):
    A = []
    for i in P:
        G = nx.erdos_renyi_graph(n,i)
        G_slovar = nx.to_dict_of_lists(G)
        A += [G_slovar]
    return A


# Ista vozlišča in verjetnost, večkrat


In [6]:
A = erdos_renyi(50, 0.3, 30)

In [7]:
np1000_nakljucno = []
casi_np1000_nakljucno = []
resitve = []
for i in A:
    time1 = time()
    nak = nakljucni_MIS(nx.Graph(i), list(np.random.permutation(len(nx.Graph(i).nodes))))
    time2 = time()
    np1000_nakljucno.append(len(nak))
    casi_np1000_nakljucno.append(time2 - time1)
    resitve.append(nak)

In [8]:
np1000_CLP = []
casi_np1000_CLP = []
for i in A:
    X = Graph()
    from_dict_of_lists(X, i)
    time1 = time()
    rez = CLP(X)
    time2 = time()
    np1000_CLP.append(len(rez))
    casi_np1000_CLP.append(time2 - time1)

In [9]:
np1000_lokalno = []
casi_np1000_lokalno = []
for i in range(len(A)):
    time1 = time()
    rez = lokalno_iskanje(nx.Graph(A[i]), resitve[i])
    time2 = time()
    np1000_lokalno.append(len(rez))
    casi_np1000_lokalno.append(time2 - time1)

In [10]:
data_np1000 = {"Nakljucno":np1000_nakljucno, "Casi nakljucno": casi_np1000_nakljucno,
           "CLP": np1000_CLP, "Casi CLP": casi_np1000_CLP ,
            "Lokalno iskanje": np1000_lokalno, "Casi lokalno": casi_np1000_lokalno}
data_np1000 = pd.DataFrame(data_np1000)



# Različna vozlišča, ista verjetnost

In [52]:
N = list(range(1, 601, 10))
B = erdos_renyi_N(N, 0.005)

In [53]:
Np_nakljucno = []
casi_Np_nakljucno = []
resitve = []
for i in B:
    time1 = time()
    nak = nakljucni_MIS(nx.Graph(i), list(np.random.permutation(len(nx.Graph(i).nodes))))
    time2 = time()
    Np_nakljucno.append(len(nak))
    casi_Np_nakljucno.append(time2 - time1)
    resitve.append(nak)

In [54]:
Np_CLP = []
casi_Np_CLP = []
for i in B:
    X = Graph()
    from_dict_of_lists(X, i)
    time1 = time()
    rez = CLP(X)
    time2 = time()
    Np_CLP.append(len(rez))
    casi_Np_CLP.append(time2 - time1)

In [55]:
Np_lokalno = []
casi_Np_lokalno = []
for i in range(len(B)):
    time1 = time()
    rez = lokalno_iskanje(nx.Graph(B[i]), resitve[i])
    time2 = time()
    Np_lokalno.append(len(rez))
    casi_Np_lokalno.append(time2 - time1)

In [57]:
data_Np = {"Nakljucno":Np_nakljucno, "Casi nakljucno": casi_Np_nakljucno,
           "CLP": Np_CLP, "Casi CLP": casi_Np_CLP ,
            "Lokalno iskanje": Np_lokalno, "Casi lokalno": casi_Np_lokalno}
data_Np = pd.DataFrame(data_Np)

# Ista vozlišča, različna verjetnost

In [103]:
P = list(np.linspace(0.1, 0.8, 50))
C = erdos_renyi_P(30, P)

In [104]:
nP_nakljucno = []
casi_nP_nakljucno = []
resitve = []
for i in C:
    time1 = time()
    nak = nakljucni_MIS(nx.Graph(i), list(np.random.permutation(len(nx.Graph(i).nodes))))
    time2 = time()
    nP_nakljucno.append(len(nak))
    casi_nP_nakljucno.append(time2 - time1)
    resitve.append(nak)

In [105]:
nP_CLP = []
casi_nP_CLP = []
for i in C:
    X = Graph()
    from_dict_of_lists(X, i)
    time1 = time()
    rez = CLP(X)
    time2 = time()
    nP_CLP.append(len(rez))
    casi_nP_CLP.append(time2 - time1)

In [106]:
nP_lokalno = []
casi_nP_lokalno = []
for i in range(len(C)):
    time1 = time()
    rez = lokalno_iskanje(nx.Graph(C[i]), resitve[i])
    time2 = time()
    nP_lokalno.append(len(rez))
    casi_nP_lokalno.append(time2 - time1)

In [107]:
data_nP = {"Nakljucno":nP_nakljucno, "Casi nakljucno": casi_nP_nakljucno,
           "CLP": nP_CLP, "Casi CLP": casi_nP_CLP ,
            "Lokalno iskanje": nP_lokalno, "Casi lokalno": casi_nP_lokalno}
data_nP = pd.DataFrame(data_nP)
print(data_nP)



    Nakljucno  Casi nakljucno  CLP  Casi CLP  Lokalno iskanje  Casi lokalno
0          13        0.000899   15  0.003948               13      0.000274
1          12        0.000827   15  0.002712               12      0.000226
2          11        0.001216   14  0.003637               11      0.000257
3          12        0.001164   13  0.003627               12      0.000269
4          10        0.001313   12  0.004509               11      0.000269
5          12        0.001091   15  0.003103               12      0.000296
6          11        0.000927   12  0.004152               11      0.000247
7           8        0.000962   12  0.004660                9      0.000283
8           9        0.001111   11  0.008904                9      0.000292
9           8        0.001186   12  0.004833                9      0.000427
10          9        0.001735   10  0.009955                9      0.000314
11          8        0.001010   10  0.006809                8      0.000303
12          

In [110]:
zapisi_v_json("grafi_ver",C)
data_nP.to_csv('grafi_ver.csv')

In [111]:
zapisi_v_json("grafi1000", A)
data_np1000.to_csv('grafi1000.csv')

In [112]:
zapisi_v_json("grafi_voz",B)
data_Np.to_csv('grafi_voz.csv')